In [1]:
import sympy as sp

cT, Cl, Cd, Wi, Wf = sp.symbols("c_T C_L C_D W_i W_f")
cl_rat = sp.symbols("\\frac{C_L^{1/2}}{C_D}")
E, R, S, rho = sp.symbols("E R S \\rho_\\infty")

In [2]:
rang = sp.Eq(
    R,
    (2/cT) * sp.sqrt(2 / (rho*S)) * cl_rat * (sp.sqrt(Wi)-sp.sqrt(Wf))
)
endur = sp.Eq(
    E,
    (1/cT) * Cl / Cd * sp.log(Wi/Wf)
)
display(rang)
display(endur)

Eq(R, 2*sqrt(2)*\frac{C_L^{1/2}}{C_D}*sqrt(1/(S*\rho_\infty))*(-sqrt(W_f) + sqrt(W_i))/c_T)

Eq(E, C_L*log(W_i/W_f)/(C_D*c_T))

In [3]:
cT_eq = sp.solve(rang,cT)[0]
display(cT_eq)

2*sqrt(2)*\frac{C_L^{1/2}}{C_D}*sqrt(1/(S*\rho_\infty))*(-sqrt(W_f) + sqrt(W_i))/R

In [4]:
cT_num = sp.lambdify([cl_rat,S,rho,Wf,Wi,R],cT_eq,'numpy')
E_num = sp.lambdify([Cl,Cd,cT,Wf,Wi],endur,'numpy')

In [5]:
AR = 10 #wing is too small, Cl is too big.
b = 300
S  = b**2 / AR
print(S)
rho_inf = 7.38e-4
V = 775 # ft / s
W = 2e6

Cd_0 = 0.03 #get it under .025 to .02
e = 0.95 #might be a little high, start with .9

# start with high aspect ratio
Cd_func  = lambda Cl : Cd_0 + Cl**2 / (3.14159 * e * AR)

Cl_init = 2*W / (rho_inf * V**2 * S)
Cd_init = Cd_func(Cl_init)
print(Cl_init,Cd_init, Cl_init**0.5/Cd_init)

Cl_final = 2*(W-1e6) / (rho_inf * V**2 * S)
Cd_final = Cd_func(Cl_final)
print(Cl_final,Cd_final,Cl_final**0.5/Cd_final)

ratio_averages = 0.5 * (Cl_init**0.5/Cd_init + Cl_final**0.5/Cd_final)
print(ratio_averages)

9000.0
1.002669293660216 0.06368544732709355 15.72311728035463
0.501334646830108 0.03842136183177339 18.428547461996857
17.075832371175743


In [6]:
range_ft = 11507 * 5280 #converting miles to ft
range_estimate = cT_num(ratio_averages,S,rho_inf,1e6,2e6,range_ft) *3600
print(range_estimate)

0.45994745534250586


In [7]:
bleh = E_num(0.5*(Cl_init + Cl_final),0.5*(Cd_init + Cd_final),range_estimate,1e6,2e6)
print(bleh)

False


In [8]:
import numpy as np
from scipy.optimize import fsolve

mu = 0.02
gross_takeoff_weight = W = 2e6
rho_0 = 2.376e-3
Clmax = 1.8 # 2.5 is too high
V_stall = np.sqrt(2*W / (Clmax * rho_0 * S))
print(f"V_stall: {V_stall}")
def ground_roll(T):
    V1 = (1.2 * V_stall)
    D = Cd_0 * 0.5 * rho_0 * V1**2 * S
    F_S = T - mu * W
    F_1 = T - D
    S_GR = W / 2 / 32.33 * V1**2 /  (F_S-F_1) * np.log(F_S / F_1)
    return S_GR

out = fsolve(lambda T: ground_roll(T) - 1e4, x0 = 1e5)
print(out)

V_stall: 322.3660295475886
[506877.5727383]
